In [1]:
# Cell 1: Imports
import numpy as np
import pandas as pd
import tensorflow as tf
import keras_nlp
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

2025-03-12 18:49:41.235043: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-12 18:49:41.253217: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741785581.272347   57413 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741785581.277912   57413 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-12 18:49:41.298662: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [6]:
# Cell 2: Load and Preprocess Data
# Load your email dataset
df = pd.read_csv('/home/shasank/shasank/ml/projects/email-classifier/dataSets/toy_dataSet.csv')

df.head()
df.rename(columns={'Category': 'label','Message': 'email_text'}, inplace=True)

In [7]:

# Encode labels
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(df['label'])
num_classes = len(label_encoder.classes_)

# Split data
X_train, X_test, y_train, y_test = train_test_split(
    df['email_text'].values, 
    encoded_labels,
    test_size=0.2, 
    random_state=42
)

In [8]:
# Cell 3: Create and Configure BERT Classifier
# Initialize the classifier
classifier = keras_nlp.models.BertClassifier.from_preset(
    "bert_base_en",
    num_classes=num_classes
)

# Compile the model
classifier.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

2025-03-12 18:52:02.541804: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [ ]:
'''# Check CUDA and GPU
nvidia-smi

# Create virtual environment
python -m venv bert_env
source bert_env/bin/activate

# Install dependencies
pip install tensorflow==2.15.0 keras-nlp==0.8.0 pandas==2.2.0 numpy==1.24.0 scikit-learn==1.4.0'''

In [ ]:
# Memory and performance optimizations
import tensorflow as tf
import os

# Set memory growth
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)

# Optimize performance
tf.config.optimizer.set_jit(True)  # Enable XLA optimization
AUTOTUNE = tf.data.AUTOTUNE

# Dataset parameters
BATCH_SIZE = 16  # Smaller batch size for large dataset
BUFFER_SIZE = 1000
MAX_LENGTH = 128

# Create tf.data pipeline
def create_dataset(texts, labels):
    dataset = tf.data.Dataset.from_tensor_slices((texts, labels))
    dataset = dataset.shuffle(BUFFER_SIZE)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(AUTOTUNE)
    return dataset

In [ ]:
# Memory and performance optimizations
import tensorflow as tf
import os

# Set memory growth
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)

# Optimize performance
tf.config.optimizer.set_jit(True)  # Enable XLA optimization
AUTOTUNE = tf.data.AUTOTUNE

# Dataset parameters
BATCH_SIZE = 16  # Smaller batch size for large dataset
BUFFER_SIZE = 1000
MAX_LENGTH = 128

# Create tf.data pipeline
def create_dataset(texts, labels):
    dataset = tf.data.Dataset.from_tensor_slices((texts, labels))
    dataset = dataset.shuffle(BUFFER_SIZE)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(AUTOTUNE)
    return dataset

In [ ]:
# Training setup with gradient accumulation
strategy = tf.distribute.MirroredStrategy()

with strategy.scope():
    classifier = keras_nlp.models.BertClassifier.from_preset(
        "bert_base_en",
        num_classes=num_classes
    )
    
    classifier.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5),
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
    )

# Training parameters
callbacks = [
    tf.keras.callbacks.ModelCheckpoint(
        filepath='checkpoints/bert_model_{epoch}.keras',
        save_best_only=True,
        monitor='val_accuracy'
    ),
    tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=2,
        restore_best_weights=True
    ),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.2,
        patience=1
    )
]

In [ ]:
'''# Monitor GPU usage (run in separate terminal)
watch -n 2 nvidia-smi

# Monitor system resources
htop'''

In [ ]:
# Cell 4: Train the Model with Updated Callbacks
callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=3,
        restore_best_weights=True
    ),
    tf.keras.callbacks.ModelCheckpoint(
        filepath='best_model.keras',  # Changed from .h5 to .keras
        monitor='val_accuracy',
        save_best_only=True
    )
]

# Train the model
history = classifier.fit(
    X_train,
    y_train,
    validation_split=0.2,
    epochs=10,
    batch_size=32,
    callbacks=callbacks
)

Epoch 1/10


2025-03-12 18:53:00.447399: E tensorflow/core/util/util.cc:131] oneDNN supports DT_INT64 only on platforms with AVX-512. Falling back to the default Eigen-based implementation if present.


In [ ]:
# Cell 5: Evaluate and Visualize
import matplotlib.pyplot as plt

# Evaluate
test_results = classifier.evaluate(X_test, y_test)
print(f"Test accuracy: {test_results[1]:.4f}")

# Plot training history
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()